# Instalacja potrzebnych pakietów, importowanie pakietów i wczytanie danych

In [ ]:
!pip install scikit-optimize
!pip install autogluon
!pip install bottleneck


In [ ]:
#po wykonaniu tego trzeba zrestartować sesję!!

# # 1. uninstall all affected packages
!pip uninstall -y Cython scipy pyparsing scikit_learn imbalanced-learn mlxtend yellowbrick

# # 2. install packages to be downgraded
!pip install Cython==0.29.36 scipy==1.9 pyparsing==2.4

# # 3. install older scikit-learn disregarding its dependencies
!pip install scikit-learn==0.24.2 --no-build-isolation

# # 4. finally install auto-sklearn
!pip install auto-sklearn

Found existing installation: scipy 1.11.4
Uninstalling scipy-1.11.4:
  Successfully uninstalled scipy-1.11.4
Found existing installation: pyparsing 2.4.0
Uninstalling pyparsing-2.4.0:
  Successfully uninstalled pyparsing-2.4.0
Found existing installation: scikit-learn 0.24.2
Uninstalling scikit-learn-0.24.2:
  Successfully uninstalled scikit-learn-0.24.2
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
  Using cached scipy-1.9.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (43.9 MB)
  Using cached pyparsing-2.4.0-py2.py3-none-any.whl (62 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
auto-sklearn 0.15.0 requires scikit-learn<0.25.0,>=0.24.0, which is not installed.
autogluon-core 1.0.0 requires scikit-learn<1.5,>=1.3.0, which is not installed.
autogluon-multimodal 1.0.0 re

  Using cached scikit_learn-0.24.2-cp310-cp310-linux_x86_64.whl
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-core 1.0.0 requires scikit-learn<1.5,>=1.3.0, but you have scikit-learn 0.24.2 which is incompatible.
autogluon-features 1.0.0 requires scikit-learn<1.5,>=1.3.0, but you have scikit-learn 0.24.2 which is incompatible.
autogluon-multimodal 1.0.0 requires scikit-learn<1.5,>=1.3.0, but you have scikit-learn 0.24.2 which is incompatible.
autogluon-tabular 1.0.0 requires scikit-learn<1.5,>=1.3.0, but you have scikit-learn 0.24.2 which is incompatible.
bigframes 0.18.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.1.4 which is incompatible.
bigframes 0.18.0 requires scikit-learn>=1.2.2, but you have scikit-learn 0.24.2 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
import random
from skopt import BayesSearchCV
from sklearn.metrics import balanced_accuracy_score
random.seed(2023)
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFE, SequentialFeatureSelector
from sklearn.inspection import permutation_importance
from xgboost.plotting import plot_importance
from autosklearn.classification import AutoSklearnClassifier
from autosklearn.experimental.askl2 import AutoSklearn2Classifier
from autosklearn.metrics import roc_auc, balanced_accuracy
from pprint import pprint
import sklearn.metrics
import sklearn.model_selection
from sklearn.metrics import balanced_accuracy_score
import autosklearn.classification
import autosklearn.metrics
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from autogluon.tabular import TabularPredictor
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
#wczytanie danych
url = "artificial_train.data"
X = pd.read_csv(url, header=None, sep=' ')

url = "artificial_train.labels"
y = pd.read_csv(url, header=None)

url = "artificial_test.data"
X_test_final1 = pd.read_csv(url, header=None, sep=' ')

#Wstępna eksploracja danych

In [ ]:
X.shape

(2000, 501)

In [ ]:
X = X.iloc[:, :500] #ostatnia kolumna jest pusta, więc ją usuwamy

In [ ]:
#sprawdzenie typów zmiennych

np.unique(X.dtypes)

#wszystkie zmienne są tego samego typu: int64

array([dtype('int64')], dtype=object)

In [ ]:
#liczby unikalnych wartości we wszystkich kolumnach
for c in X.columns:
  print(f"{c}: {len(np.unique(X[c].values))}")

0: 40
1: 170
2: 208
3: 60
4: 222
5: 44
6: 213
7: 11
8: 92
9: 47
10: 196
11: 160
12: 183
13: 227
14: 76
15: 130
16: 80
17: 169
18: 130
19: 64
20: 81
21: 126
22: 44
23: 63
24: 227
25: 157
26: 125
27: 75
28: 75
29: 133
30: 52
31: 194
32: 194
33: 211
34: 141
35: 219
36: 72
37: 136
38: 32
39: 12
40: 24
41: 209
42: 130
43: 97
44: 160
45: 137
46: 196
47: 183
48: 200
49: 146
50: 229
51: 197
52: 124
53: 34
54: 160
55: 122
56: 218
57: 186
58: 217
59: 122
60: 112
61: 43
62: 205
63: 44
64: 402
65: 197
66: 199
67: 128
68: 34
69: 45
70: 224
71: 89
72: 134
73: 201
74: 48
75: 219
76: 62
77: 142
78: 207
79: 115
80: 141
81: 218
82: 167
83: 83
84: 210
85: 173
86: 35
87: 119
88: 44
89: 226
90: 5
91: 186
92: 125
93: 113
94: 51
95: 180
96: 25
97: 132
98: 112
99: 191
100: 58
101: 58
102: 45
103: 85
104: 192
105: 552
106: 155
107: 180
108: 177
109: 68
110: 174
111: 221
112: 51
113: 141
114: 196
115: 102
116: 118
117: 75
118: 155
119: 133
120: 16
121: 137
122: 67
123: 212
124: 127
125: 185
126: 79
127: 139
128

In [ ]:
#które kolumny mają ile braków danych
X.isna().sum()[X.isna().sum()!=0] #nie ma braków danych

Series([], dtype: int64)

In [ ]:
#podział zbioru na część trenignową i testową
#dodajemy stratyfikację na y, aby zachować balans w obu klasach

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, test_size = 0.3, random_state=203, stratify=y)

In [ ]:
#sprawdzamy pary zmiennych, które mają korelację >0.95 lub <-0.95 i jedą ze zmiennych z każdej takiej pary usuwamy

c = X.iloc[:,:-1].corr().abs()

#tylko dolna część macierzy
mask = np.triu(np.ones(c.shape), k=1).astype(bool)
s = c.where(mask).unstack()

result = s[np.abs(s) >= 0.95]
print(result.to_string())

data_result = result.index.to_list()
duze_corr = pd.DataFrame(data_result, columns=['Zmienna 1', 'Zmienna 2'])
do_usuniecia = duze_corr['Zmienna 2']
do_usuniecia = do_usuniecia.drop_duplicates()
X = X.drop(columns=do_usuniecia)
X_train = X_train.drop(columns=do_usuniecia)
X_test = X_test.drop(columns=do_usuniecia)

do_usuniecia # w ten sposób usunęłyśmy 12 zmiennych


#'reset' indeksów kolumn
X.columns = range(len(X.columns))
X_train.columns = range(len(X_train.columns))
X_test.columns = range(len(X_test.columns))

128  105    0.989410
281  153    0.988695
318  28     0.989628
336  64     0.990429
378  48     0.988338
433  153    0.989355
     281    0.989047
451  28     0.989168
     318    0.988690
472  442    0.990300
475  241    0.989272
493  453    0.988458


### dane testowe

In [ ]:
X_test_final1 = X_test_final1.iloc[:, :500] #ostatnia kolumna jest pusta, więc ją usuwamy

In [ ]:
print("Braki danych: ", X_test_final1.isna().sum()[X_test_final1.isna().sum()!=0])
print("Typy kolumn: ", np.unique(X_test_final1.dtypes))

Braki danych:  Series([], dtype: int64)
Typy kolumn:  [dtype('int64')]


In [ ]:
#usuwamy te same kolumny co wcześniej
X_test_final = X_test_final1.drop(columns=do_usuniecia)
X_test_final.columns = range(len(X_test_final.columns))

# Modele ręczne

## bez selekcji *zmiennych*

### XGBoost

In [ ]:
#xgb wymaga klas 0 i 1 a u nas są -1 i 1, więc zamieniamy wartości
y_train = y_train.replace(-1, 0)
y_test = y_test.replace(-1,0)

#### RandomizedSearchCV

In [ ]:
m_xgb = XGBClassifier(random_state=2023, n_jobs=-1)

params_xgb = {
    'max_depth': (4, 10),
    'learning_rate': (0.01, 1.0),
    'n_estimators': (100, 500),
    #'subsample': (0.1, 1.0, 'uniform'),
    #'colsample_bytree': (0.1, 1.0, 'uniform'),
    'gamma': (0, 1.0),
    'min_child_weight': (1, 10),
    'reg_alpha': (0, 1.0),
    'reg_lambda': (0, 1.0),
    #'scale_pos_weight': (1, 10),
    #'max_delta_step': (0, 10),
    #'booster': ['gbtree', 'gblinear', 'dart'],
    'n_jobs': [-1],
}


xgb_rs = RandomizedSearchCV(estimator=m_xgb, param_distributions=params_xgb, refit=True, cv=3, verbose=1, scoring='balanced_accuracy', random_state=2023)


xgb_rs.fit(X_train, y_train)

print("Balanced accuracy score = ", balanced_accuracy_score(y_test, xgb_rs.predict(X_test)))
#Balanced accuracy score =  0.805

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Balanced accuracy score =  0.805


#### Optymalizacja bayesowska

In [ ]:
#optymalizacja bayesowska :)))

cv_strategy = KFold(n_splits=3, shuffle=True, random_state=2023)

params = {
    'max_depth': (4, 10),
    'learning_rate': (0.01, 1.0, 'log-uniform'),
    'n_estimators': (100, 1000),
    'gamma': (0, 1.0),
    'min_child_weight': (1, 10),
    'reg_alpha': (0, 1.0),
    'reg_lambda': (0, 1.0),
    'n_jobs': [-1],
    'max_delta_step': (0, 10),
    'subsample': (0.1, 1.0, 'uniform'),
    'colsample_bytree': (0.1, 1.0, 'uniform'),
}

xgboost_model = XGBClassifier(random_state=2023)
opt = BayesSearchCV(xgboost_model, params, n_iter=100, cv=cv_strategy, verbose=1, random_state=2023, scoring='balanced_accuracy')
_ = opt.fit(X_train, y_train)
# Get the best parameters
best_params = opt.best_params_
print("Best Parameters:", best_params)
print(opt.score(X_train, y_train))
bs_cb_results_click = opt.cv_results_

#Best Parameters: OrderedDict([('colsample_bytree', 0.7634941189516811), ('gamma', 1.0), ('learning_rate', 0.01), ('max_delta_step', 0), ('max_depth', 10), ('min_child_weight', 1), ('n_estimators', 100), ('n_jobs', -1), ('reg_alpha', 0.053698404016961146), ('reg_lambda', 0.0), ('subsample', 1.0)])

print("Balanced accuracy score = ", balanced_accuracy_score(y_test, opt.predict(X_test)))
#0.8316666666666668

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

### Las losowy


#### RandomizedSearchCV

In [ ]:
#Kroswalidacja

m = RandomForestClassifier(random_state=2023, n_jobs=-1)

params = {
    "n_estimators": [100,200,300,400,500],
    "max_depth": [4,5,6,7,8,9,10,12,15,20],
    "min_samples_split": [2,3,4,5,6,7,8,9,10],
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'max_features': ['auto', 'sqrt', 'log2', None],
    'bootstrap': [True, False]

}

m_rs = RandomizedSearchCV(estimator=m, param_distributions=params, refit=True, cv=3, verbose=1, scoring='balanced_accuracy', random_state=2023)

m_rs.fit(X_train, y_train.values.ravel())

print("Wybrane najlepsze parametry:\n", m_rs.best_params_)
#{'n_estimators': 400, 'min_samples_split': 7, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 6, 'bootstrap': False}

print("Balanced accuracy score = ", balanced_accuracy_score(y_test, m_rs.predict(X_test)))
#Balanced accuracy score =  0.7949999999999999

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Wybrane najlepsze parametry:
 {'n_estimators': 400, 'min_samples_split': 7, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 6, 'bootstrap': False}
Balanced accuracy score =  0.7949999999999999


#### Optymalizacja bayesowska

In [ ]:
#optymalizacja bayesowska :)))

cv_strategy = KFold(n_splits=3, shuffle=True, random_state=2023)

params = {
    "n_estimators": [100,200,300,400,500],
    "max_depth": [4,5,6,7,8,9,10,12,15,20],
    "min_samples_split": [2,3,4,5,6,7,8,9,10],
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'max_features': ['auto', 'sqrt', 'log2', None],
    'bootstrap': [True, False]
}

model_rf = RandomForestClassifier(random_state=2023, n_jobs=-1)
opt_model_rf = BayesSearchCV(model_rf, params, n_iter=100, cv=cv_strategy, verbose=1, random_state=2023, scoring='balanced_accuracy')
_ = opt_model_rf.fit(X_train, y_train.values.ravel())
# Get the best parameters
best_params = opt_model_rf.best_params_
print("Best Parameters:", best_params)
print(opt_model_rf.score(X_train, y_train.values.ravel()))
bs_cb_results_click = opt_model_rf.cv_results_

#Best Parameters: OrderedDict([('bootstrap', True), ('max_depth', 12), ('max_features', None), ('min_samples_leaf', 1), ('min_samples_split', 5), ('n_estimators', 200)])

print("Balanced accuracy score = ", balanced_accuracy_score(y_test, opt_model_rf.predict(X_test))) #jest troche inny wynik niz wyzej
#Balanced accuracy score =  0.825

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Best Parameters: OrderedDict([('bootstrap', True), ('max_depth', 9), ('max_features', None), ('min_samples_leaf', 4), ('min_samples_split', 10), ('n_estimators', 500)])
0.9928571428571429
Balanced accuracy score =  0.825


### ExtraTrees

#### RandomizedSearchCV

In [ ]:
m_et = ExtraTreesClassifier(random_state=2023, n_jobs=-1)

param_space = {
    'n_estimators': (10, 700),
    'max_features': (0.1, 1.0),
    'min_samples_split': (2, 20),
    'min_samples_leaf': (1, 20),
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy'],

}

et_rs = RandomizedSearchCV(estimator=m_et, param_distributions=param_space, refit=True, cv=3, verbose=1, scoring='balanced_accuracy', random_state=2023)


et_rs.fit(X_train, y_train.values.ravel())

print("Balanced accuracy score = ", balanced_accuracy_score(y_test, et_rs.predict(X_test)))
#Balanced accuracy score = 0.8400001

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Balanced accuracy score =  0.8400000000000001


#### Optymalizacja bayesowska

In [ ]:
cv_strategy = KFold(n_splits=3, shuffle=True, random_state=2023)

param_space = {
    'n_estimators': (10, 200),         # Number of trees in the forest
    'max_features': (0.1, 1.0),        # Proportion of features to consider for split
    'min_samples_split': (2, 20),      # Minimum number of samples required to split an internal node
    'min_samples_leaf': (1, 20),       # Minimum number of samples required to be at a leaf node
    'bootstrap': [True, False],        # Whether to bootstrap samples
    'criterion': ['gini', 'entropy']   # Split criterion
}

extratrees_model = ExtraTreesClassifier(random_state=2023)
opt = BayesSearchCV(extratrees_model, param_space, n_iter=100, cv=cv_strategy, verbose=1, random_state=2023, scoring='balanced_accuracy')
_ = opt.fit(X_train, y_train.values.ravel())
# Get the best parameters
best_params = opt.best_params_
print("Best Parameters:", best_params)
print(opt.score(X_train, y_train.values.ravel()))
bs_cb_results_click = opt.cv_results_

print("Balanced accuracy score = ", balanced_accuracy_score(y_test, opt.predict(X_test)))

#Balanced accuracy score =  0.8416666666666667

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits


/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 3 folds for each of 1 candidates, totalling 3 fits


/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Best Parameters: OrderedDict([('bootstrap', False), ('criterion', 'entropy'), ('max_features', 1.0), ('min_samples_leaf', 1), ('min_samples_split', 2), ('n_estimators', 151)])
1.0
Balanced accuracy score =  0.8416666666666667


## selekcja RFE (Recursive Feature Elimination)

In [ ]:
#XGB
xgb = XGBClassifier(random_state=2023)
selector = RFE(xgb, step=1)
selector.fit(X_train, y_train)
selected_variables_xgb_rfe = X_train.columns[selector.support_]
print("Selected Variables:", selected_variables_xgb_rfe)


#Random Forest
rf = RandomForestClassifier(random_state=2023)
selector = RFE(rf, step=1)
selector.fit(X_train, y_train.values.ravel())
selected_variables_rf_rfe = X_train.columns[selector.support_]
print("Selected Variables:", selected_variables_rf_rfe)


#ExtraTrees
et = ExtraTreesClassifier(random_state=2023)
selector = RFE(et, step=1)
selector.fit(X_train, y_train.values.ravel())
selected_variables_et_rfe = X_train.columns[selector.support_]
print("Selected Variables:", selected_variables_et_rfe)

Selected Variables: Index([  2,   4,   6,  10,  11,  13,  15,  16,  20,  21,
       ...
       476, 477, 479, 480, 482, 483, 485, 486, 487, 489],
      dtype='int64', length=245)
Selected Variables: Index([  2,   4,  10,  12,  13,  17,  18,  24,  25,  26,
       ...
       473, 474, 475, 477, 480, 482, 483, 484, 488, 489],
      dtype='int64', length=245)
Selected Variables: Index([  2,   4,   5,  10,  11,  12,  13,  15,  16,  18,
       ...
       474, 476, 480, 481, 482, 483, 485, 486, 487, 488],
      dtype='int64', length=245)


### XGBoost

#### RandomizedSearchCV

In [ ]:
m_xgb = XGBClassifier(random_state=2023)

params_xgb = {
    'max_depth': (4, 10),
    'learning_rate': (0.01, 1.0),
    'n_estimators': (100, 700),
    #'subsample': (0.1, 1.0, 'uniform'),
    #'colsample_bytree': (0.1, 1.0, 'uniform'),
    'gamma': (0, 1.0),
    'min_child_weight': (1, 10),
    'reg_alpha': (0, 1.0),
    'reg_lambda': (0, 1.0),
    #'scale_pos_weight': (1, 10),
    #'max_delta_step': (0, 10),
    #'booster': ['gbtree', 'gblinear', 'dart'],
    'n_jobs': [-1],
}

xgb_rs = RandomizedSearchCV(estimator=m_xgb, param_distributions=params_xgb, refit=True, cv=3, verbose=1, scoring='balanced_accuracy', random_state=2023)


xgb_rs.fit(X_train.loc[:, selected_variables_xgb_rfe], y_train)

print("Balanced accuracy score = ", balanced_accuracy_score(y_test, xgb_rs.predict(X_test.loc[:, selected_variables_xgb_rfe])))
#Balanced accuracy score =  0.8083333333333333

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Balanced accuracy score =  0.8083333333333333


#### Optymalizacja bayesowska

In [ ]:
cv_strategy = KFold(n_splits=3, shuffle=True, random_state=2023)

params = {
    'max_depth': (4, 10),
    'learning_rate': (0.01, 1.0, 'log-uniform'),
    'n_estimators': (100, 1000),
    'gamma': (0, 1.0),
    'min_child_weight': (1, 10),
    'reg_alpha': (0, 1.0),
    'reg_lambda': (0, 1.0),
    'n_jobs': [-1],
    'max_delta_step': (0, 10),
    'subsample': (0.1, 1.0, 'uniform'),
    'colsample_bytree': (0.1, 1.0, 'uniform'),
}

xgboost_model = XGBClassifier(random_state=2023)
opt = BayesSearchCV(xgboost_model, params, n_iter=100, cv=cv_strategy, verbose=1, random_state=2023, scoring='balanced_accuracy')
_ = opt.fit(X_train.loc[:, selected_variables_xgb_rfe], y_train)
# Get the best parameters
best_params = opt.best_params_
print("Best Parameters:", best_params)
print(opt.score(X_train.loc[:, selected_variables_xgb_rfe], y_train))
bs_cb_results_click = opt.cv_results_

print("Balanced accuracy score = ", balanced_accuracy_score(y_test, opt.predict(X_test.loc[:, selected_variables_xgb_rfe])))
#0.85

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

### Las losowy


#### RandomizedSearchCV

In [ ]:
#Kroswalidacja

m = RandomForestClassifier(random_state=2023, n_jobs=-1)

params = {
    "n_estimators": [100,200,300,400,500],
    "max_depth": [4,5,6,7,8,9,10,12,15,20],
    "min_samples_split": [2,3,4,5,6,7,8,9,10],
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'max_features': ['auto', 'sqrt', 'log2', None],
    'bootstrap': [True, False]

}

m_rs_rfe = RandomizedSearchCV(estimator=m, param_distributions=params, refit=True, cv=3, verbose=1, scoring='balanced_accuracy', random_state=2023)

m_rs_rfe.fit(X_train.loc[:, selected_variables_rf_rfe], y_train.values.ravel())

print("Wybrane najlepsze parametry:\n", m_rs_rfe.best_params_)

print("Balanced accuracy score = ", balanced_accuracy_score(y_test, m_rs_rfe.predict(X_test.loc[:, selected_variables_rf_rfe])))

#{'n_estimators': 400, 'min_samples_split': 7, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 6, 'bootstrap': False}
#Balanced accuracy score =  0.7983333333333333

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Wybrane najlepsze parametry:
 {'n_estimators': 400, 'min_samples_split': 7, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 6, 'bootstrap': False}
Balanced accuracy score =  0.7983333333333333


#### Optymalizacja bayesowska

In [ ]:
#optymalizacja bayesowska

cv_strategy = KFold(n_splits=3, shuffle=True, random_state=2023)

params = {
    'max_depth': (4, 10),
    'learning_rate': (0.01, 1.0, 'log-uniform'),
    'n_estimators': (100, 1000),
    'gamma': (0, 1.0),
    'min_child_weight': (1, 10),
    'reg_alpha': (0, 1.0),
    'reg_lambda': (0, 1.0),
    'n_jobs': [-1],
    'max_delta_step': (0, 10),
    'subsample': (0.1, 1.0, 'uniform'),
    'colsample_bytree': (0.1, 1.0, 'uniform'),
}

xgboost_model = XGBClassifier(random_state=2023)
opt = BayesSearchCV(xgboost_model, params, n_iter=100, cv=cv_strategy, verbose=1, random_state=2023, scoring='balanced_accuracy')
_ = opt.fit(X_train, y_train)
# Get the best parameters
best_params = opt.best_params_
print("Best Parameters:", best_params)
print(opt.score(X_train, y_train))
bs_cb_results_click = opt.cv_results_

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

### ExtraTrees

#### RandomizedSearchCV

In [ ]:
m_et = ExtraTreesClassifier(random_state=2023)


param_space = {
    'n_estimators': (10, 700),
    'max_features': (0.1, 1.0),
    'min_samples_split': (2, 20),
    'min_samples_leaf': (1, 20),
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy'],

}


et_rs = RandomizedSearchCV(estimator=m_et, param_distributions=param_space, refit=True, cv=3, verbose=1, scoring='balanced_accuracy', random_state=2023)


et_rs.fit(X_train.loc[:, selected_variables_et_rfe], y_train.values.ravel())

print("Balanced accuracy score = ", balanced_accuracy_score(y_test, et_rs.predict(X_test.loc[:, selected_variables_et_rfe])))
#Balanced accuracy score =  0.8516666666666666

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Balanced accuracy score =  0.8516666666666666


#### Optymalizacja bayesowska

In [ ]:
cv_strategy = KFold(n_splits=3, shuffle=True, random_state=2023)

param_space = {
    'n_estimators': (10, 200),         # Number of trees in the forest
    'max_features': (0.1, 1.0),        # Proportion of features to consider for split
    'min_samples_split': (2, 20),      # Minimum number of samples required to split an internal node
    'min_samples_leaf': (1, 20),       # Minimum number of samples required to be at a leaf node
    'bootstrap': [True, False],        # Whether to bootstrap samples
    'criterion': ['gini', 'entropy']   # Split criterion
}

extratrees_model = ExtraTreesClassifier(random_state=2023)
opt = BayesSearchCV(extratrees_model, param_space, n_iter=100, cv=cv_strategy, verbose=1, random_state=2023, scoring='balanced_accuracy')
_ = opt.fit(X_train.loc[:, selected_variables_et_rfe], y_train.values.ravel())
# Get the best parameters
best_params = opt.best_params_
print("Best Parameters:", best_params)
print(opt.score(X_train.loc[:, selected_variables_et_rfe], y_train.values.ravel()))
bs_cb_results_click = opt.cv_results_

print("Balanced accuracy score = ", balanced_accuracy_score(y_test, opt.predict(X_test.loc[:, selected_variables_et_rfe])))

#Balanced accuracy score = 0.8400000000000001

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits


/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Best Parameters: OrderedDict([('bootstrap', False), ('criterion', 'gini'), ('max_features', 0.9953718310286855), ('min_samples_leaf', 1), ('min_samples_split', 8), ('n_estimators', 190)])
1.0
Balanced accuracy score =  0.8400000000000001


## selekcja bazująza na informacji wzajemnej

####potrzebna klasa

In [ ]:
from scipy import signal
from sklearn.utils import check_X_y, check_array
from sklearn.preprocessing import StandardScaler
from multiprocessing import cpu_count
from sklearn.base import BaseEstimator
import bottleneck as bn
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
from sklearn.feature_selection import SelectorMixin

NUM_CORES = cpu_count()
class MutualInformationFeatureSelector(BaseEstimator, SelectorMixin):
    def __init__(self, method='JMI', k=5, n_features='auto', categorical=True, n_jobs=1, verbose=0, threshold=0.0):
        self.method = method
        self.k = k
        self.n_features = n_features
        self.categorical = categorical
        self.n_jobs = n_jobs
        self.verbose = verbose
        self.threshold = threshold
        self._support_mask = None

    def _get_support_mask(self):
        if self._support_mask is None:
            raise ValueError('Feature selector has not been fitted yet!')
        return self._support_mask

    def _isinteger(self, x):
        return np.all(np.equal(np.mod(x, 1), 0))

    def _check_params(self, X, y):
        X, y = check_X_y(X, y)

        if not self.categorical:
            ss = StandardScaler()
            X = ss.fit_transform(X)
            y = ss.fit_transform(y.reshape(-1, 1))
        if self.categorical and np.any(self.k > np.bincount(y)):
            raise ValueError('k must be smaller than your smallest class.')
        return X, y

    def _add_remove(self, S, F, i):
        S.append(i)
        F.remove(i)
        return S, F

    def _print_results(self, S, MIs):
        out = ''
        if self.n_features == 'auto':
            out += 'Auto selected feature #' + str(len(S)) + ' : ' + str(S[-1])
        else:
            out += ('Selected feature #' + str(len(S)) + ' / ' + str(self.n_features) + ' : ' + str(S[-1]))

        if self.verbose > 1:
            out += ', MI : ' + str(MIs[-1])
        print(out)

    def fit(self, X, y):
        if self.n_jobs < 0:
            self.n_jobs = NUM_CORES - self.n_jobs
        self.X, y = self._check_params(X, y)
        n, p = X.shape
        self.y = y.reshape((n, 1))

        S = []
        F = list(range(p))

        if self.categorical:
            mutual_info = mutual_info_classif
        else:
            mutual_info = mutual_info_regression

        xy_MI = mutual_info(X, y.flatten())
        S, F = self._add_remove(S, F, bn.nanargmax(xy_MI))
        S_mi = [bn.nanmax(xy_MI)]

        if self.verbose > 0:
            self._print_results(S, S_mi)

        if self.n_features == 'auto':
            n_features = np.inf
        else:
            n_features = self.n_features

        while len(S) < n_features:
            s = len(S) - 1
            remaining_X = self.X[:, F]
            mi_values = mutual_info(remaining_X, self.y.flatten())

            if len(mi_values) == 0 or np.max(mi_values) < self.threshold:
                break

            if self.method == 'JMI':
                selected = F[np.argmax(mi_values)]
            elif self.method == 'JMIM':
                if bn.allnan(mi_values):
                    break
                selected = F[np.nanargmax(mi_values)]
            elif self.method == 'MRMR':
                if bn.allnan(mi_values):
                    break
                MRMR = xy_MI[F] - bn.nanmean(mi_values, axis=0)
                selected = F[np.nanargmax(MRMR)]

            S, F = self._add_remove(S, F, selected)
            S_mi.append(np.max(mi_values))

            if self.verbose > 0:
                self._print_results(S, S_mi)

        self.n_features_ = len(S)
        self._support_mask = np.zeros(p, dtype=bool)
        self._support_mask[S] = True
        self.ranking_ = S
        self.mi_ = S_mi

        return self

#### Selekcja

In [ ]:
feat_selector = feat_selector = MutualInformationFeatureSelector(method='JMIM', k=3, n_features='auto', categorical=False, verbose=1, threshold=0.015)
feat_selector.fit(X_train, y_train.values.ravel())
print(feat_selector._support_mask)
print(feat_selector.ranking_)

#selekcja
X_train_filtered = feat_selector.transform(X_train)
print(X_train_filtered.shape)
X_test_filtered=X_test.loc[:,feat_selector._support_mask]

### XGBoost

#### RandomizedSearchCV

In [ ]:
m_xgb = XGBClassifier(random_state=2023, n_jobs=-1)

params_xgb = {
    'max_depth': (4, 10),
    'learning_rate': (0.01, 1.0),
    'n_estimators': (100, 500),
    #'subsample': (0.1, 1.0, 'uniform'),
    #'colsample_bytree': (0.1, 1.0, 'uniform'),
    'gamma': (0, 1.0),
    'min_child_weight': (1, 10),
    'reg_alpha': (0, 1.0),
    'reg_lambda': (0, 1.0),
    #'scale_pos_weight': (1, 10),
    #'max_delta_step': (0, 10),
    #'booster': ['gbtree', 'gblinear', 'dart'],
    'n_jobs': [-1],
}


xgb_rs = RandomizedSearchCV(estimator=m_xgb, param_distributions=params_xgb, refit=True, cv=3, verbose=1, scoring='balanced_accuracy', random_state=2023)


xgb_rs.fit(X_train_filtered, y_train)

print("Balanced accuracy score = ", balanced_accuracy_score(y_test, xgb_rs.predict(X_test_filtered)))
#0.7533333333333334

#### Optymalizacja bayesowska

In [ ]:
cv_strategy = KFold(n_splits=3, shuffle=True, random_state=2023)

params_xgb = {
    'max_depth': (4, 10),
    'learning_rate': (0.01, 1.0),
    'n_estimators': (100, 500),
    #'subsample': (0.1, 1.0, 'uniform'),
    #'colsample_bytree': (0.1, 1.0, 'uniform'),
    'gamma': (0, 1.0),
    'min_child_weight': (1, 10),
    'reg_alpha': (0, 1.0),
    'reg_lambda': (0, 1.0),
    #'scale_pos_weight': (1, 10),
    #'max_delta_step': (0, 10),
    #'booster': ['gbtree', 'gblinear', 'dart'],
    'n_jobs': [-1],
}

model_xgb = XGBClassifier(random_state=2023, n_jobs=-1)
opt_model_xgb = BayesSearchCV(model_xgb, params_xgb, n_iter=100, cv=cv_strategy, verbose=1, random_state=2023, scoring='balanced_accuracy')
_ = opt_model_xgb.fit(X_train_filtered, y_train.values.ravel())
# Get the best parameters
best_params = opt_model_xgb.best_params_
print("Best Parameters:", best_params)
print(opt_model_xgb.score(X_train_filtered, y_train.values.ravel()))
bs_cb_results_click = opt_model_xgb.cv_results_

print("Balanced accuracy score = ", balanced_accuracy_score(y_test, opt_model_xgb.predict(X_test_filtered)))
#0.827

### Las losowy


#### RandomizedSearchCV

In [ ]:
las = RandomForestClassifier(random_state=2023, n_jobs=-1)

params = {
    "n_estimators": [100,200,300,400,500],
    "max_depth": [4,5,6,7,8,9,10,12,15,20],
    "min_samples_split": [2,3,4,5,6,7,8,9,10],
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'max_features': ['auto', 'sqrt', 'log2', None],
    'bootstrap': [True, False]
}

m_rs = RandomizedSearchCV(estimator=las, param_distributions=params, refit=True, cv=3, verbose=1, scoring='balanced_accuracy', random_state=202)

m_rs.fit(X_train_filtered,y_train)

print("Wybrane najlepsze parametry:\n", m_rs.best_params_)
print("Balanced accuracy score = ", balanced_accuracy_score(y_test, m_rs.predict(X_test_filtered)))
#0.825

#### Optymalizacja bayesowska

In [ ]:
cv_strategy = KFold(n_splits=3, shuffle=True, random_state=2023)

params = {
    "n_estimators": [100,200,300,400,500],
    "max_depth": [4,5,6,7,8,9,10,12,15,20],
    "min_samples_split": [2,3,4,5,6,7,8,9,10],
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'max_features': ['auto', 'sqrt', 'log2', None],
    'bootstrap': [True, False]
}

las = RandomForestClassifier(random_state=2023, n_jobs=-1)
opt_rf= BayesSearchCV(las, params, n_iter=100, cv=cv_strategy, verbose=1, random_state=2023, scoring='balanced_accuracy')
_ = opt_rf.fit(X_train_filtered, y_train.values.ravel())
# Get the best parameters
best_params = opt_rf.best_params_
print("Best Parameters:", best_params)
print(opt_rf.score(X_train_filtered, y_train.values.ravel()))
bs_cb_results_click = opt_rf.cv_results_

print("Balanced accuracy score = ", balanced_accuracy_score(y_test, opt_rf.predict(X_test_filtered)))
#0.8166666666666667

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


### ExtraTrees

#### RandomizedSearchCV

In [ ]:
et = ExtraTreesClassifier(random_state=2023, n_jobs=-1)

params = {
    "n_estimators": [50,100,200,300,400,500],
    "max_depth": [4,5,6,7,8,9,10,12,15,20],
    "min_samples_split": [2,3,4,5,6,7,8,9,10],
    'min_samples_leaf': [1, 2, 3, 4, 5, 7, 10],
    'max_features': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy']
}

et_rs = RandomizedSearchCV(estimator=et, param_distributions=params, refit=True, cv=3, verbose=1, scoring='balanced_accuracy', random_state=2023)

et_rs.fit(X_train_filtered,y_train.values.ravel())

print("Wybrane najlepsze parametry:\n", et_rs.best_params_)

print("Balanced accuracy score = ", balanced_accuracy_score(y_test, et_rs.predict(X_test_filtered)))
#0.745

#### Optymalizacja bayesowska

In [ ]:
cv_strategy = KFold(n_splits=3, shuffle=True, random_state=2023)

param_space = {
    'n_estimators': (10, 200),         # Number of trees in the forest
    'max_features': (0.1, 1.0),        # Proportion of features to consider for split
    'min_samples_split': (2, 20),      # Minimum number of samples required to split an internal node
    'min_samples_leaf': (1, 20),       # Minimum number of samples required to be at a leaf node
    'bootstrap': [True, False],        # Whether to bootstrap samples
    'criterion': ['gini', 'entropy']   # Split criterion
}

extratrees_model = ExtraTreesClassifier(random_state=2023)
opt = BayesSearchCV(extratrees_model, param_space, n_iter=100, cv=cv_strategy, verbose=1, random_state=2023, scoring='balanced_accuracy')
_ = opt.fit(X_train_filtered, y_train.values.ravel())
# Get the best parameters
best_params = opt.best_params_
print("Best Parameters:", best_params)
print("Balanced accuracy score = ", balanced_accuracy_score(y_test, opt.predict(X_test_filtered))) #0.765

# Modele AutoMLowe

## AutoSklearn

In [ ]:
settings = {
  "time_left_for_this_task": 300,
  "seed": 2023,
  "metric": balanced_accuracy,
  "n_jobs": 4,
  "initial_configurations_via_metalearning": 70
  #"initial_configurations_via_metalearning": 75 => test = 0.8300000000000001
  #"ensemble_size":
}

# This will only be used by autosklearn 1 while autosklearn 2 will automaticallybselect a strategy
#resampling_strategy = "holdout"
resampling_strategy = "cv-iterative-fit"
#resampling_strategy_arguments={"folds": 3}

# Create and train an ensemble with AutoML
# Auto-sklearn will ingest the pandas dataframe and detects column types
askl1 = AutoSklearnClassifier(
    **settings,
    resampling_strategy=resampling_strategy,
    #resampling_strategy_arguments=resampling_strategy_arguments,
)

askl1.fit(X_train, y_train)

In [ ]:
def scoring_function(estimator):
        pred = estimator.predict(X_test)
        #pred = estimator.predict_proba(X_test)[:, 1]
        #pred[pred>=0.5] = 1
        #pred[pred<0.5] = 0
        return sklearn.metrics.balanced_accuracy_score(y_test, pred)
def train_scoring_function(estimator):
        pred = estimator.predict(X_train)
        #pred = estimator.predict_proba(X_train)[:, 1]
        #pred[pred>=0.5] = 1
        #pred[pred<0.5] = 0
        return sklearn.metrics.balanced_accuracy_score(y_train, pred)


In [ ]:
leaderboard = askl1.leaderboard(sort_by="model_id")
print(leaderboard)
print(f"Auto-sklearn 1.0 | train = {train_scoring_function(askl1)} | test = {scoring_function(askl1)}")
print(f"Selected `resampling-strategy` = {askl1.resampling_strategy}")
print(f"Selected `resampling-strategy-arguments` = {askl1.resampling_strategy_arguments}")

print(askl1.sprint_statistics())

leaderboard = askl1.leaderboard(sort_by="model_id")
print(leaderboard)

pprint(askl1.show_models())
#test = 0.8366666666666667

In [ ]:
predictions = askl1.predict_proba(X_test_final)

class_1_probabilities = predictions[:, 1]

final_data = pd.DataFrame({'Probabilities': ['323145_306011'] + list(class_1_probabilities)})

# Zapisanie do pliku txt
final_data.to_csv('predictions_with_label_autosklearn.txt', index=False, header=False)

## AutoGluon

In [ ]:
data = X_train.copy()
data['label'] = y_train
label = 'label'

#predictor = TabularPredictor(label=label).fit(data)
predictor = TabularPredictor(label=label, eval_metric='balanced_accuracy',problem_type='binary').fit(data)

predictions = predictor.predict_proba(X_test)

class_1_probabilities = predictions.iloc[:, 1]

# Wypisanie wyników
print(class_1_probabilities)

print("Balanced accuracy score = ", balanced_accuracy_score(y_test, predictor.predict(X_test)))

#Balanced accuracy score =  0.8366666666666667

In [ ]:
predictions_final = predictor.predict_proba(X_test_final)

class_1_probabilities_final = predictions_final.iloc[:, 1]

final_data = pd.DataFrame({'Probabilities': ['323145_306011'] + list(class_1_probabilities_final)})

# Zapisanie do pliku txt
final_data.to_csv('predictions_with_label.txt', index=False, header=False)